In [178]:
import numpy as np

def gen_employees(number_employees, number_shifts):
    
    return [[np.random.random() + .5 for i in range(number_shifts)] for j in range(number_employees)]
    
    
def submission(values_by_employee):
    
    return [[ values_by_employee[i][j] < 1 for j in range( len(values_by_employee[0]) ) ] for i in range( len(values_by_employee) ) ] 


def rank_desired_shifts(submissions):
    
    shift_submissions = [0]*len(submissions[0])
    
    for j in range(len(submissions)):
        for i in range(len(submissions[0])):
            if submissions[j][i]:
                shift_submissions[i]+=1
    sorted_index_pos = [index for index, num in sorted(enumerate(shift_submissions), key=lambda x: x[-1])]
    return(sorted_index_pos)
    
    
def rank_employees(submissions):
    
    employees_submissions = [0]*len(submissions)
    
    for i in range(len(submissions)):
        for j in range(len(submissions[0])):
            if submissions[i][j]:
                employees_submissions[i]+=1
    sorted_index_pos = [index for index, num in sorted(enumerate(employees_submissions), key=lambda x: x[-1])]
    return(sorted_index_pos)


def availability_window(submissions, shifts_ranked, employees_ranked, capacity):
    
    shifts_selected = []
    assignments = [[0 for i in range(len(shifts_ranked)+3)] for j in range(len(employees_ranked)+1)]
    assignments[0][len(assignments[0])-1] = "Wages"
    assignments[0][len(assignments[0])-2] = "Assignment"

    for i in range(len(assignments)-1):
        assignments[i+1][0] = "E" + str(i)
        assignments[i+1][len(assignments[0])-2] = []
    for j in range(len(assignments[0])-3):
        assignments[0][j+1] = "S" + str(j)
    
    for i in range(len(submissions)):
        for j in range(len(submissions[0])):
            if submissions[i][j]:
                assignments[i+1][j+1]="Y"
            else: 
                assignments[i+1][j+1]="N"   
    
    employee_counts = [capacity]*len(employees_ranked)

    for shift in shifts_ranked:
        unselected = True
        for employee in employees_ranked:
            if submissions[employee][shift] and unselected and employee_counts[employee] > 0:
                assignments[employee+1][len(assignments[0])-2].append("S"+ str(shift))
                unselected = False
                employee_counts[employee]-=1
                shifts_selected.append(shift)
                continue
            continue
    for i in range(len(assignments)-1):
        assignments[i+1][len(assignments[0])-1] = float(len(assignments[i+1][len(assignments[0])-2]))
    assignments[0][0] = ""
    
    return assignments, shifts_selected


def bid_at_value(values_by_employee, assignments, shifts_selected, number_shifts):
    
    shifts_unselected = []
    for shift in range(number_shifts):
        if shift not in shifts_selected:
            shifts_unselected.append(shift)
    
    temp=10 # arbitrary value higher than all possible values
    current_employee = -1
    print("\n" + "Shifts that went unselected")
    print(shifts_unselected)
    for shift in shifts_unselected:
        for i in range(len(values_by_employee)):
            if values_by_employee[i][shift] < temp and values_by_employee[i][shift] > 1:
                temp = values_by_employee[i][shift]
                current_employee=i
        assignments[current_employee+1][len(assignments[0])-1]+=temp
        assignments[current_employee+1][len(assignments[0])-2].append("S"+ str(shift))
    
    return assignments


def main():
    
    capacity = 2 # number of shifts per employee
    number_employees = np.random.randint(4,10) # fully scalable
    number_shifts = np.random.randint(number_employees, number_employees * capacity)
    
    values_by_employee = gen_employees(number_employees, number_shifts)
    submissions = submission(values_by_employee)
    
#     print(number_shifts)
#     print(len(submissions[0]))

#     print(len(submissions))
#     print(number_employees)
        
    shifts_ranked = rank_desired_shifts(submissions)
    print("\n"+"Shifts Ranked")
    print(shifts_ranked)
    
    employees_ranked = rank_employees(submissions)
    print("\n"+"Employees Ranked") 
    print(employees_ranked)
    
    print("\n")
    assignments, shifts_selected = availability_window(submissions, shifts_ranked, employees_ranked, capacity)
    print(np.matrix(assignments))

    print("\n" + "Availability Window assignment")
    print(shifts_selected)
    
    print("\n" + "Employee values of shifts, [0.5, 1.5]")
    print(np.matrix(values_by_employee))

    full_schedule = bid_at_value(values_by_employee, assignments, shifts_selected, number_shifts)
    print("\n" + "Incentive Window assignment, where employees bid value")
    print(np.matrix(full_schedule))
    
main()


Shifts Ranked
[0, 4, 5, 1, 2, 3]

Employees Ranked
[0, 1, 3, 2]


[['' 'S0' 'S1' 'S2' 'S3' 'S4' 'S5' 'Assignment' 'Wages']
 ['E0' 'N' 'Y' 'N' 'N' 'Y' 'N' list(['S4', 'S1']) 2.0]
 ['E1' 'N' 'N' 'Y' 'Y' 'N' 'N' list(['S2', 'S3']) 2.0]
 ['E2' 'N' 'Y' 'Y' 'Y' 'Y' 'Y' list([]) 0.0]
 ['E3' 'N' 'Y' 'Y' 'Y' 'N' 'Y' list(['S5']) 1.0]]

Availability Window assignment
[4, 5, 1, 2, 3]

Employee values of shifts, [0.5, 1.5]
[[1.45710369 0.94279498 1.19599175 1.49489754 0.53723532 1.0045374 ]
 [1.00388043 1.35885931 0.69411272 0.75739372 1.39039981 1.26472732]
 [1.33984602 0.87791539 0.80602165 0.52624372 0.63784554 0.52965657]
 [1.45327404 0.68084357 0.65268672 0.71219601 1.39399361 0.91316434]]

Shifts that went unselected
[0]

Incentive Window assignment, where employees bid value
[['' 'S0' 'S1' 'S2' 'S3' 'S4' 'S5' 'Assignment' 'Wages']
 ['E0' 'N' 'Y' 'N' 'N' 'Y' 'N' list(['S4', 'S1']) 2.0]
 ['E1' 'N' 'N' 'Y' 'Y' 'N' 'N' list(['S2', 'S3', 'S0']) 3.00388043158538]
 ['E2' 'N' 'Y' 'Y' 'Y' 'Y' 'Y' l